In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

In [ ]:
def generate_markdown_table(data):
    max_lengths = {key: max(map(len, map(str, data[key]))) for key in data.keys()}

    header = "| " + " | ".join(key.ljust(max_lengths[key]) for key in data.keys()) + " |"

    separator = "| " + " | ".join(["-" * max_lengths[key] for key in data.keys()]) + " |"

    rows = []
    for i in range(len(data["Character"])):
        row = "| " + " | ".join(str(data[key][i]).ljust(max_lengths[key]) for key in data.keys()) + " |"
        rows.append(row)

    markdown_table = "{}\n{}\n{}".format(header, separator, '\n'.join(rows))

    return markdown_table

data = {
    "Character": ["Scorpion", "Sub-Zero", "Liu Kang", "Sonya Blade", "Raiden", "Johnny Cage", "Kitana", "Kano", "Jax Briggs", "Mileena", "Shang Tsung", "Kung Lao", "Shao Kahn", "Cassie Cage", "Kotal Kahn", "Erron Black", "Fujin", "D'Vorah", "Geras", "Cetrion", "Noob Saibot", "Frost", "Kollector", "Jade", "Baraka", "Kabal", "Kung Jin", "Shinnok", "Rain", "Sareena", "Tremor"],
    "Gender": ["Male", "Male", "Male", "Female", "Male", "Male", "Female", "Male", "Male", "Female", "Male", "Male", "Male", "Female", "Male", "Male", "Male", "Female", "Male", "Female", "Male", "Female", "Male", "Male", "Male", "Male", "Male", "Female", "Male","Male","Female"],
    "Faction": ["Shirai Ryu", "Lin Kuei", "White Lotus Society", "Special Forces", "Elder Gods", "Earthrealm Warriors", "Edenia", "Black Dragon", "Special Forces", "Shao Kahn's Army", "Sorcerer", "White Lotus Society", "Outworld", "Special Forces", "Outworld", "Outworld", "Elder Gods", "Outworld", "Kronika's Keepers", "Elder Gods", "Netherrealm", "Lin Kuei", "Shao Kahn's Army", "Edenia", "Tarkatan Horde", "Black Dragon", "White Lotus Society", "Netherrealm", "Edenia", "Netherrealm", "Black Dragon"],
    "Debut Game": ["Mortal Kombat","Mortal Kombat","Mortal Kombat","Mortal Kombat","Mortal Kombat","Mortal Kombat","Mortal Kombat 2","Mortal Kombat","Mortal Kombat 2","Mortal Kombat 2","Mortal Kombat","Mortal Kombat 2","Mortal Kombat 2","Mortal Kombat 10","Mortal Kombat 10","Mortal Kombat 10","Mortal Kombat 4","Mortal Kombat 10","Mortal Kombat 11","Mortal Kombat 11","Mortal Kombat 2","Mortal Kombat: Deadly Alliance","Mortal Kombat 11","Mortal Kombat 2","Mortal Kombat 2","Mortal Kombat 3","Mortal Kombat 10","Mortal Kombat 4","Mortal Kombat 3","Mortal Kombat Mythologies: Sub-Zero","Mortal Kombat: Special Forces"],
    "Status": ["Undead","Alive","Alive","Alive","Immortal","Alive","Alive","Alive","Alive","Deceased","Alive","Alive","Deceased","Alive","Alive","Alive","Alive","Alive","Alive","Alive","Undead","Alive","Alive","Alive","Alive","Alive","Alive","Deceased","Alive","Alive","Alive"],
    "Alliance/Enemies": ["Sub Zero,Quan Chi","Scorpion,Lin Kuei","Kung Lao,Raiden","Jax Briggs,Johnny Cage","Liu Kang,Fujin","Sonya Blade,Cassie Cage","Liu Kang,Jade","Sonya Blade,Jax Briggs","Sonya Blade,Kano ","Kitana,Baraka","Quan Chi,Shao Kahn","Liu Kang,Raidem","Kitana,Mileena","Sonya Blade,Johnny Cage","D'Vorah,Mileena","Kotakl Kahn,Shang Tsung","Raiden,Liu Kang","Kotal Kahn,Mileena","Kronika,Liu Kang","Kronika,Shinnok","Sub Zero,Scorpion","Sub Zero,Quan Chi","Shao Kahn,D'Vorah","Kitana,Liu Kang","Mileena,D'Vorah","Sonya Blade,Kano","Kung Lao,Raiden","Raiden,Quan Chi","Kitana,Tanya","Quan Chi,SubZero","Kano, Jax Briggs"],
    "Weapon": ["Kunai and Spear","Ice Sword","Nuchaku,fire powers","military equipment","staff,lightning powers","Brass Knuckles,shadow moves","Steel fans","Cybernetic heart","bionic arms","Dual Sai","Magic","Razor-rimmed hat","Wrath Hammer","Pistols,Military Training","Macuahuitl,Blood Magic","Revolvers,Sand Grenades","Crossbow,Sword,Wind Powers","bladed arms","Gauntlets,Sand Powers","Elemental Powers","Shadows,Teleportation","Ice Weapons,Cybernetics","Spiked Chain,Collector Bag","Bo Staff,Polearn","Arm Blades","Dual Hooked Swords","Bo Staff,Bow","Amulet of Shinnok","Katar,Water Manipulation","Dual Kamas","Gauntlets,Earth Manipulation"],
    "Origin": ["Earthrealm","Earthrealm","Earthrealm","Earthrealm","Elder God","Earthrealm","Edenia","Earthrealm","Earthrealm","Edenia/Tarkatan","Earthrealm","Earthrealm","Outworld","Earthrealm","Outworld","Earthrealm","Elder God","Outworld","Unknown","Elder God","Earthrealm","Earthrealm","Outworld","Edenia","Outworld","Edenia","Outworld","Earthrealm","Earthrealm","Unknown","Edenia"],
    "Occupation": ["Ninja/Assassin","Lin Kuei Grandmaster","Martial Artist","Special Forces agent","Protector of Earthrealn","Actor","Princess","Black Dragon Leader","Special Forces Agent","Assassin","Sorcerer","MArtial Artist","Emperor of Outworld","Special Forces Agent","Mercenary","Protector of Earthrealn","Kytinn Warrior","agent of Kronika","Protector of Earthrealn","Wraith Assassin","Lin Kuei Warrior","Item Collector","Bodyguard","Tarkatan Warrior","Mercenary","Archer,MArtial Artist","Dark Sorcerer","Edenian Prince","Edenian Prince","Demon assassin","Black Dragon Soldier"],
    "Trademark": ["'Get Over here!'","'Spine Rip'","'Dragon Fatality'","'Kiss of Death'","'Lightning Fatality'","'Nut Punch'","'Fan Lift'","'Heart Rip'","'Heart Rip'","'Ground Pound'","'Man Eater'","'Morphing'","'Hat trick'","'Annihilation'","'Bubble Head'","'God Ray'","'Sand Trap'","'Tornado'","'Bug Burst'","'Phasing Through Time'","'Wrath of the Titans'","'Shadow Takedown'","'Icequake'","For the Kollection'","'Staff Swipe'","'Chop CHop Blades'","'Nomad Dash'","'Shaolin Drop'","'Impersonator'","'Superkick'","'demon Slam'"],
    "Affiliation": ["None","Lin Kuei","White Lotus society","Special Forces","Elder Gods","Earthrealn Warriors","Edenian royalty","Black Dragon","Special Forces","Shao Kahn's Army","Netherrealn","white Lotus Society","Outworld","Special Forces","Outwrold","Outworld/black Dragon","Elder Gods","Outworld","Kronika's Keepers","Elder Gods","Netherrealn","Lin Kuei","Shao Kahn's army","Edenian royalty","Tarkatan Horde","Former Black Dragon","White Lotus Society","Netherrealn","Edenia","Netherrealn","Black Dragon"],
    "Costume/Outfit": ["Yellow ninja garb","Blue ninja garb","Red martial arts attire","Green military attire","white and Blue attire","Black and green attire","Blue ninja attire","Metal faceplate,cybernetic","Cybernetic arms","Pink ninja attire","red robe with a skull","Red monk attire","Skull faced helmet","Military inspired attire","Aztec inspired armor","Cowboy inspired attire","White and blue attire","Insect like armor","Sand themed attire","Nature inspired attire","Dark ninja attire","Cybernetic ninja attire","Multi armed attire","Green ninja attire","Tarkatan Inspired armor","Hooked mask,respirator","Archer inspired attire","dark robes,skull mask","Purple ninja attire","Black and red attire","Brown and orange attire"],

}

markdown_table = generate_markdown_table(data)

print(markdown_table)


| Character   | Gender | Faction             | Debut Game                          | Status   | Alliance/Enemies        | Weapon                       | Origin          | Occupation              | Trademark              | Affiliation           | Costume/Outfit             |
| ----------- | ------ | ------------------- | ----------------------------------- | -------- | ----------------------- | ---------------------------- | --------------- | ----------------------- | ---------------------- | --------------------- | -------------------------- |
| Scorpion    | Male   | Shirai Ryu          | Mortal Kombat                       | Undead   | Sub Zero,Quan Chi       | Kunai and Spear              | Earthrealm      | Ninja/Assassin          | 'Get Over here!'       | None                  | Yellow ninja garb          |
| Sub-Zero    | Male   | Lin Kuei            | Mortal Kombat                       | Alive    | Scorpion,Lin Kuei       | Ice Sword                    | Earthrealm      | Lin 

In [ ]:

csv_filename = "character_data.csv"

with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    writer.writerow(data.keys())

    # Write data
    for i in range(len(data["Character"])):
        row_data = [data[key][i] for key in data.keys()]
        writer.writerow(row_data)

print(f"Data saved to {csv_filename}")


Data saved to character_data.csv
